# Demo Analysis

## 1. Setup Workspace

In [1]:
import json
import pathlib
import copy
import operator
import pathlib
import os

import anytree
import networkx as nx
import sympy

import paptree

## 2. Load Data

In [2]:
data_dir = pathlib.Path.cwd().parent / "experiments/demo/data"
trace_file = data_dir / "paptrace.darwin-arm64-clang.json"
trees = paptree.utils.from_file(trace_file)
print(f"Loaded {len(trees)} traces.")

Loaded 16 traces.


In [3]:
known_file = data_dir / "known_exprs.json"
with open(known_file, "r") as f_in:
    known_exprs = json.loads(f_in.read())
print(f"Loaded {len(known_exprs)} known expressions.")
print(json.dumps(known_exprs, indent=2))

Loaded 1 known expressions.
{
  "int operator*=(int, int)": "C_OP_MUL_ASSIGN_INT_INT"
}


## 3. Node Summary

### 3.1. Unfiltered Nodes

In [4]:
nodes = []
for tree in trees:
    nodes.extend(anytree.PreOrderIter(tree.root))
print(f"Node count: {len(nodes)}")

Node count: 163


In [5]:
node_types = {}
for node in nodes:
    node_types.setdefault(node.type, []).append(node)
print(f"Node type count: {len(node_types)}")

Node type count: 6


In [6]:
print("Nodes per type:")
for k, v in node_types.items():
    print(f"- {k}: {len(v)}")

Nodes per type:
- CalleeExpr: 16
- ReturnStmt: 16
- IfThenStmt: 2
- ForStmt: 6
- LoopIter: 41
- OpExpr: 82


### 3.2. Unique Nodes

In [7]:
unique_nodes = []
for node in nodes:
    if node not in unique_nodes:
        unique_nodes.append(node)
print(f"Unique node count: {len(unique_nodes)}")

Unique node count: 24


In [8]:
# We expect this count to match the unfiltered node type count.
unique_node_types = {}
for node in unique_nodes:
    unique_node_types.setdefault(node.type, []).append(node)
print(f"Node type count: {len(unique_node_types)}")

Node type count: 6


In [9]:
print("Nodes per type:")
for k, v in unique_node_types.items():
    print(f"- {k}: {len(v)}")

Nodes per type:
- CalleeExpr: 16
- ReturnStmt: 3
- IfThenStmt: 1
- ForStmt: 1
- LoopIter: 1
- OpExpr: 2


## 4. Trace Summary

In [10]:
binned_trees = {}
for tree in trees:
    binned_trees.setdefault(tree.root.sig, []).append(tree)
print(f"Trace entry point count: {len(binned_trees)}")

Trace entry point count: 2


In [11]:
print("Traces per entry point:")
for k, v in binned_trees.items():
    print(f"- {k}: {len(v)}")

Traces per entry point:
- _Bool demo::IsEven(int): 8
- int demo::Factorial(int): 8


## 5. Trace Analysis

### 5.1. IsEven

In [12]:
trees_subset = binned_trees["_Bool demo::IsEven(int)"]

def to_params_str(params):
    #return ", ".join([f"{param['name']}={param['value']}" for param in params])
    return ", ".join([f"{param['value']}" for param in params])

def to_call_str(node):
    return f"{node.sig}: ({to_params_str(node.params)})"

def to_simple_node_view(node):
    sym = "sym @ " if hasattr(node, "target") else ""
    if node.is_call_node():
        node_type = type(node).__name__
        desc = f"{node.type}: {to_call_str(node)}"
    else:
        node_type = type(node).__name__
        desc = f"{node.type}: {node.desc}"
    return f"({sym}{node.name}) [{node_type}] {desc}"

for tree in operator.itemgetter(0, 1, 2)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(1474) [CallNode] CalleeExpr: _Bool demo::IsEven(int): (0)
└── (1472) [StmtNode] ReturnStmt: return value % 2 == 0

(1474) [CallNode] CalleeExpr: _Bool demo::IsEven(int): (2)
└── (1472) [StmtNode] ReturnStmt: return value % 2 == 0

(1474) [CallNode] CalleeExpr: _Bool demo::IsEven(int): (4)
└── (1472) [StmtNode] ReturnStmt: return value % 2 == 0



In [13]:
results = paptree.analyze.analyze(known_exprs, trees_subset)
print("\nResults:")
print(json.dumps(results, indent=2))

Path summary:
- _Bool demo::IsEven(int): 1 paths
  - [path_0] (1472,): 8 traces

Finding general expr. for: _Bool demo::IsEven(int): (0)
  Found expr.: C_1474

Results:
{
  "sigs": {
    "_Bool demo::IsEven(int)": "sig_0"
  },
  "ctxs": {
    "sig_0": {
      "_Bool demo::IsEven(int): (0)": 0,
      "_Bool demo::IsEven(int): (2)": 0,
      "_Bool demo::IsEven(int): (4)": 0,
      "_Bool demo::IsEven(int): (1)": 0,
      "_Bool demo::IsEven(int): (3)": 0,
      "_Bool demo::IsEven(int): (5)": 0,
      "_Bool demo::IsEven(int): (-2147483648)": 0,
      "_Bool demo::IsEven(int): (2147483647)": 0
    }
  },
  "exprs": {
    "sig_0": {
      "0": "Symbol('C_1474')"
    }
  }
}


### 5.2. Factorial

In [14]:
trees_subset = binned_trees["int demo::Factorial(int)"]
for tree in operator.itemgetter(0, 2, 3, 4)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(1678) [CallNode] CalleeExpr: int demo::Factorial(int): (-1)
└── (1564) [StmtNode] IfThenStmt: value < 0 || value > 31
    └── (1562) [StmtNode] ReturnStmt: return -1

(1678) [CallNode] CalleeExpr: int demo::Factorial(int): (0)
├── (1662) [LoopNode] ForStmt: for (int i = 1; i <= value; ++i)
└── (1676) [StmtNode] ReturnStmt: return result

(1678) [CallNode] CalleeExpr: int demo::Factorial(int): (1)
├── (1662) [LoopNode] ForStmt: for (int i = 1; i <= value; ++i)
│   ├── (1662) [StmtNode] LoopIter: LoopIter
│   │   └── (1653) [StmtNode] OpExpr: int operator*=(int, int)
│   └── (1639) [StmtNode] OpExpr: int operator++
└── (1676) [StmtNode] ReturnStmt: return result

(1678) [CallNode] CalleeExpr: int demo::Factorial(int): (2)
├── (1662) [LoopNode] ForStmt: for (int i = 1; i <= value; ++i)
│   ├── (1662) [StmtNode] LoopIter: LoopIter
│   │   └── (1653) [StmtNode] OpExpr: int operator*=(int, int)
│   ├── (1639) [StmtNode] OpExpr: int operator++
│   ├── (1662) [StmtNode] LoopIter: LoopIter
│  

In [15]:
results = paptree.analyze.analyze(known_exprs, trees_subset)
print("\nResults:")
print(json.dumps(results, indent=2))

Path summary:
- int demo::Factorial(int): 3 paths
  - [path_0] (1564, 1562): 2 traces
  - [path_1] (1662, 1676): 1 traces
  - [path_2] (1662, 1662, 1676): 5 traces

Finding general expr. for: int demo::Factorial(int): (0)
  Found expr.: C_1678

Finding general expr. for: int demo::Factorial(int): (1)
  Found expr.: C_1678

Finding general expr. for: int demo::Factorial(int): (2)
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    15.72      4.67996e+06        4                0                0     21.69s
  Found loop expr.: sqrt(X0**2)
Found established loop expr: sqrt(X0**2)
  Found expr.: C_1678 + (C_OP_MUL_ASSIGN_INT_INT + T_1639)*sqrt(X0**2)

Results:
{
  "sigs": {
    "int demo::Factorial(int)": "sig_0"
  },
  "ctxs": {
    "sig_0": {
      "int demo::Factorial(int): (-1)": 0,
     